# PersonaryデータをExcelで取得する例

Personaryに格納されているデータをExcelファイルとして取得するためのNotebookです。

事前に、 `00_PLR初期設定.ipynb` を実施している想定です。

# 取得対象の決定

Personaryに格納されているデータを特定するため、以下の情報を設定してください。

In [ ]:
from datetime import datetime, timedelta

# チャネル名(変更不要)
channel_name = 'cog-pds-log'

# データ取得開始
# 1日前から取得する場合
since = datetime.now() - timedelta(days=1)
# 日付指定
# since = '2025-06-15'
# 日付指定なし
# since = None

# データ取得終了
# sinceと同様に相対時間、日付指定で指定可能
until = None

# タスクのURL(データ特定用)
task_url = 'https://goemon.cloud/t/uOY1fQp0yzK6zGQEdS5q'

まず、チャネルリストを取得...

In [ ]:
# JSONLデータの処理には、Pandasというライブラリを使うのが便利
import pandas as pd

# このセルを実行後、 ←のファイル一覧に、channels.jsonlが見えるはず...
!source ~/.plrprofile && plrget list -o channels.jsonl

channels = pd.read_json('channels.jsonl', orient='records', lines=True)
channels

指定されたチャネルが存在しているかのチェック(ないとエラーになります)

In [ ]:
assert len(channels[channels['absoluteName'] == channel_name]) > 0, f'No channels named "{channel_name}"'

`plrget` コマンドにより、チャネルに格納されているデータを取得します。

In [ ]:
def datetime_to_str(dt):
    """datetimeオブジェクトを文字列に変換"""
    if isinstance(dt, str):
        return dt
    if isinstance(dt, datetime):
        return dt.strftime('%Y-%m-%dT%H:%M:%S')
    # わからない型の場合はそのまま返す
    return dt

# チャネルのIDを取得
channel_id = channels[channels['absoluteName'] == channel_name]['id'].values[0]

# チャネルを指定して`plrget`コマンドを実行
options = ''
if since is not None:
    s_since = datetime_to_str(since)
    options += f' --since {s_since}'
if until is not None:
    s_until = datetime_to_str(until)
    options += f' --until {s_until}'
!source ~/.plrprofile && plrget list {options} {channel_id} -o data.jsonl

data = pd.read_json('data.jsonl', orient='records', lines=True)
data

指定されたタスクの情報を抽出します。

In [ ]:
import json

assert task_url.startswith('https://goemon.cloud/t/'), 'task_urlは https://goemon.cloud/t/ から始まる必要があります'
target_task_id = task_url[len('https://goemon.cloud/t/'):]
print('Task ID', target_task_id)

def is_target_task(cog_pds_json):
    meta = cog_pds_json['meta']
    task_id = meta['task']['id']
    return task_id == target_task_id

target_data = data[data['cogPDSJSON'].apply(lambda x: is_target_task(json.loads(x)))]
target_data

In [ ]:
target_data['cogPDSJSON'].values[0]

In [ ]:
import json
import pandas as pd

all_dfs = []

for idx, row in target_data.iterrows():
    try:
        raw_json = row["cogPDSJSON"]
        jspsych_data = json.loads(raw_json)
    except json.JSONDecodeError:
        jspsych_data = json.loads(eval(raw_json))

    trials = jspsych_data.get("data", {}).get("data", [])
    meta = jspsych_data.get("meta", {})

    df = pd.DataFrame(trials)

    # メタ情報を付加
    df["participant_id"] = row['cogPDSUser']
    df["task_title"] = meta.get("task", {}).get("title", "unknown")
    df["task_started"] = meta.get("started")
    df["task_finished"] = meta.get("finished")

    all_dfs.append(df)

# すべての行を結合
result_df = pd.concat(all_dfs, ignore_index=True)

# 欲しい順序でカラムを並べる
first_columns = [
    "participant_id",
    "task_title",
    "task_started",
    "task_finished",
    "trial_type",
    "time_elapsed",
    "rt",
    "stimulus"
]

# 存在しないカラムがある場合でも落ちないようにする
first_columns = [col for col in first_columns if col in result_df.columns]

# 残りのカラムを取得（重複排除）
other_columns = [col for col in result_df.columns if col not in first_columns]

# カラム順を設定
ordered_columns = first_columns + other_columns
result_df = result_df[ordered_columns]

# Excelに保存
result_df

In [ ]:
result_df.to_excel('data.xlsx')